# Exporting MNASNet 1_0 from Pytorch to ONNX format. 

In [19]:
import torch 
from torchvision.models import mnasnet1_0, MNASNet1_0_Weights

## Load the pre-trained model

In [20]:
weights = MNASNet1_0_Weights.IMAGENET1K_V1
mnasnet1_0 = mnasnet1_0(weights=weights)

## Provide random input tensor

In [21]:
input = torch.randn(1, 3, 224, 224)

## Export the model to onnx format

In [22]:
torch.onnx.export(mnasnet1_0,
                  input,
                  "mnasnet10.onnx",
                  input_names=["input"],
                  output_names=["output"])

## Verify ONNX Model

In [23]:
import onnxruntime as ort

alexnet_onnx = ort.InferenceSession("mnasnet10.onnx")

print("Input names:", alexnet_onnx.get_inputs()[0].name)
print("Output names:", alexnet_onnx.get_outputs()[0].name)

Input names: input
Output names: output


## Verification Using Netron


<img src="input_mnasnet10.png"  width="1000" height="400">

<img src="output_mnasnet10.png"  width="1000" height="500">

## Using ONNXRUNTIME

Implementing MNASNet_1_0 on the CPU involves severar implementation steps

- Preparation of the input tensor (preproceccing)
- Performing an inference in the CPU accelerator
- Converting class probabilities to class labels (postprocessing)

<img src="church.jpg" width="300" height="400">

In [24]:
import numpy as np
import cv2




def preprocessing(image): 
    
    img = cv2.resize(image, (256, 256))
    height, width, _ = img.shape
    x = (width - 224)//2
    y = (height - 224)//2
    img = img[y:y+224, x:x+224]
    img = np.asarray(img, np.float32)
    img = np.transpose(img, [2, 0, 1])
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]
    for i in range(3):
        img[i,:,:] = (img[i,:,:]/255 - mean[i] / std[i])
    
    img = np.expand_dims(img, axis=0)
    
    return img


img = "church.jpg"
img = cv2.imread(img)
img = preprocessing(img)

    

In [25]:
import onnxruntime as ort

session_options = ort.SessionOptions()
session = ort.InferenceSession("mnasnet10.onnx", session_options, ["CPUExecutionProvider"])


input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
ortvalue = ort.OrtValue.ortvalue_from_numpy(img)
session = session.run([output_name], {input_name:ortvalue})

In [26]:
import os 


def read_classes(filepath):
    with open(filepath, 'r') as file: 
        classes = [line.strip() for line in file]
    return classes
        



def postprocessing(data, classes):
    
    data = np.exp(data - np.max(data))
    prob = data/data.sum()
    pred_index = np.argmax(prob)
    predicted_class = classes[pred_index]
    
    return predicted_class, pred_index
    

classes = read_classes(os.path.abspath('../classes.txt'))

predicted_class, predicted_index = postprocessing(session, classes)

print("Predicted Class:", predicted_class)
print("Predicted Index:", predicted_index)
    
    

Predicted Class: church
Predicted Index: 497


Implementation of MNASNET_1_0 on the GPU. In case you installed `onnxruntime`, you need uninstall onnxruntime and install `pip install onnxruntime-gpu`. If you are facing problems with libs and failures, install `onnxruntime-gpu` from other [source](https://github.com/microsoft/onnxruntime/issues/20944). In Addition import `torch`to avoid errors. 

```bash

python3 -m pip install --index-url=https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/ onnxruntime_gpu

 ```

In [27]:
import onnxruntime as ort
import torch 

providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']



session_options = ort.SessionOptions()
session_gpu = ort.InferenceSession("mnasnet10.onnx", session_options, providers)
input_name = session_gpu.get_inputs()[0].name
output_name = session_gpu.get_outputs()[0].name
io_binding = session_gpu.io_binding()
ortvalue = ort.OrtValue.ortvalue_from_numpy(img, 'cuda', 0)
io_binding.bind_input(name=input_name,
                        device_type='cuda',
                        device_id=0,
                        element_type=np.float32,
                        shape = ortvalue.shape(),
                        buffer_ptr=ortvalue.data_ptr()
                        )

io_binding.bind_output(name= output_name,
                             device_type='cuda',
                             device_id=0)

session_gpu.run_with_iobinding(io_binding)

output = io_binding.copy_outputs_to_cpu()
predicted_class, predicted_index = postprocessing(output, classes)

print("Predicted Class:", predicted_class)
print("Predicted Index:", predicted_index)

Predicted Class: church
Predicted Index: 497
